In [28]:
import math
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

In [52]:
pd.read_csv('./dts/train.csv').drop('ID',axis=1).to_csv('train.csv',index=False)
pd.read_csv('./dts/test.csv').drop('ID',axis=1).to_csv('test.csv',index=False)

In [53]:
train_data_file = './train.csv'
test_data_file = './train.csv'

dp_train = pd.read_csv('./dts/train.csv').drop('ID',axis=1)
dp_test = pd.read_csv('./dts/test.csv').drop('ID',axis=1)

dp_train['Height(cm)'] = dp_train.pop('Height(Feet)')*30.48 + dp_train.pop('Height(Remainder_Inches)')*2.54
dp_test['Height(cm)'] = dp_test.pop('Height(Feet)')*30.48 + dp_test.pop('Height(Remainder_Inches)')*2.54


dp_train['Weight(kg)'] = dp_train.pop('Weight(lb)')*.453592
dp_test['Weight(kg)'] = dp_test.pop('Weight(lb)')*.453592

dp_train['BMI'] = dp_train['Weight(kg)']/(dp_train['Height(cm)']/100)**2
dp_test['BMI'] = dp_test['Weight(kg)']/(dp_test['Height(cm)']/100)**2

dp_train.drop('Weight_Status', axis=1, inplace=True)
dp_test.drop('Weight_Status', axis=1, inplace=True)

dp_train['Age_Status'] = (np.where(dp_train['Age']<20, 'child', np.where(dp_train['Age']<40, 'adult', np.where(dp_train['Age']<65, 'mid_adult', 'high_adult'))))
dp_test['Age_Status'] = (np.where(dp_test['Age']<20, 'child', np.where(dp_test['Age']<40, 'adult', np.where(dp_test['Age']<65, 'mid_adult', 'high_adult'))))
dp_train['Calories_Burned'] = dp_train.pop('Calories_Burned')

dp_train['fnlwgt'] = 0
dp_test['fnlwgt'] = 0
dp_test['Calories_Burned'] = 0
CSV_HEADER = dp_train.columns

print(f"Train data shape: {dp_train.shape}")
print(f"Test data shape: {dp_test.shape}")

Train data shape: (7500, 11)
Test data shape: (7500, 11)


In [54]:
NUMERICAL_FEATURE_NAMES = [
    'Exercise_Duration', 'Body_Temperature(F)', 'BPM', 'Age', 'Height(cm)',
    'Weight(kg)', 'BMI'
]

CATEGORICAL_FEATURES_WITH_VOCABULARY = {
    'Gender': sorted(list(dp_train['Gender'].unique())),
    'Age_Status': sorted(list(dp_train['Age_Status'].unique())),
}

WEIGHT_COLUMN_NAME = 'fnlwgt'

CATEGORICAL_FEATURE_NAMES = list(CATEGORICAL_FEATURES_WITH_VOCABULARY.keys())

FEATURE_NAMES = NUMERICAL_FEATURE_NAMES + CATEGORICAL_FEATURE_NAMES

COLUMN_DEFAULTS = [
    [0.0] if feature_name in NUMERICAL_FEATURE_NAMES + [WEIGHT_COLUMN_NAME] else ['NA']
    for feature_name in CSV_HEADER
]

TARGET_FEATURE_NAME = 'Calories_Burned'

In [55]:
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.0001
DROPOUT_RATE = 0.2
BATCH_SIZE = 265
NUM_EPOCHS = 10

NUM_TRANSFORMER_BLOCKS = 3  # Number of transformer blocks.
NUM_HEADS = 4  # Number of attention heads.
EMBEDDING_DIMS = 16  # Embedding dimensions of the categorical features.
MLP_HIDDEN_UNITS_FACTORS = [
    2,
    1,
]  # MLP hidden layer units, as factors of the number of inputs.
NUM_MLP_BLOCKS = 2  # Number of MLP blocks in the baseline model.

In [56]:
def prepare_example(features, target):
    weights = features.pop(WEIGHT_COLUMN_NAME)
    # return features, target_index, weights
    return features, weights


def get_dataset_from_csv(csv_file_path, batch_size=128, shuffle=False):
    dataset = tf.data.experimental.make_csv_dataset(
        csv_file_path,
        batch_size=batch_size,
        column_names=CSV_HEADER,
        column_defaults=COLUMN_DEFAULTS,
        label_name=TARGET_FEATURE_NAME,
        num_epochs=1,
        header=False,
        na_value="?",
        shuffle=shuffle,
    ).map(prepare_example, num_parallel_calls=tf.data.AUTOTUNE, deterministic=False)
    return dataset.cache()

def run_experiment(
    model,
    train_data_file,
    test_data_file,
    num_epochs,
    learning_rate,
    weight_decay,
    batch_size,
):

    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.BinaryCrossentropy(),
        metrics=[keras.metrics.BinaryAccuracy(name="accuracy")],
    )

    train_dataset = get_dataset_from_csv(train_data_file, batch_size, shuffle=True)
    validation_dataset = get_dataset_from_csv(test_data_file, batch_size)

    print("Start training the model...")
    history = model.fit(
        train_dataset, epochs=num_epochs, validation_data=validation_dataset
    )
    print("Model training finished")

    _, accuracy = model.evaluate(validation_dataset, verbose=0)

    print(f"Validation accuracy: {round(accuracy * 100, 2)}%")

    return history

def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        if feature_name in NUMERICAL_FEATURE_NAMES:
            inputs[feature_name] = layers.Input(
                name=feature_name, shape=(), dtype=tf.float32
            )
        else:
            inputs[feature_name] = layers.Input(
                name=feature_name, shape=(), dtype=tf.string
            )
    return inputs

def encode_inputs(inputs, embedding_dims):

    encoded_categorical_feature_list = []
    numerical_feature_list = []

    for feature_name in inputs:
        if feature_name in CATEGORICAL_FEATURE_NAMES:

            # Get the vocabulary of the categorical feature.
            vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY[feature_name]

            # Create a lookup to convert string values to an integer indices.
            # Since we are not using a mask token nor expecting any out of vocabulary
            # (oov) token, we set mask_token to None and  num_oov_indices to 0.
            lookup = layers.StringLookup(
                vocabulary=vocabulary,
                mask_token=None,
                num_oov_indices=0,
                output_mode="int",
            )

            # Convert the string input values into integer indices.
            encoded_feature = lookup(inputs[feature_name])

            # Create an embedding layer with the specified dimensions.
            embedding = layers.Embedding(
                input_dim=len(vocabulary), output_dim=embedding_dims
            )

            # Convert the index values to embedding representations.
            encoded_categorical_feature = embedding(encoded_feature)
            encoded_categorical_feature_list.append(encoded_categorical_feature)

        else:

            # Use the numerical features as-is.
            numerical_feature = tf.expand_dims(inputs[feature_name], -1)
            numerical_feature_list.append(numerical_feature)

    return encoded_categorical_feature_list, numerical_feature_list

def create_mlp(hidden_units, dropout_rate, activation, normalization_layer, name=None):

    mlp_layers = []
    for units in hidden_units:
        mlp_layers.append(normalization_layer),
        mlp_layers.append(layers.Dense(units, activation=activation))
        mlp_layers.append(layers.Dropout(dropout_rate))

    return keras.Sequential(mlp_layers, name=name)

def create_baseline_model(
    embedding_dims, num_mlp_blocks, mlp_hidden_units_factors, dropout_rate
):

    # Create model inputs.
    inputs = create_model_inputs()
    # encode features.
    encoded_categorical_feature_list, numerical_feature_list = encode_inputs(
        inputs, embedding_dims
    )
    # Concatenate all features.
    features = layers.concatenate(
        encoded_categorical_feature_list + numerical_feature_list
    )
    # Compute Feedforward layer units.
    feedforward_units = [features.shape[-1]]

    # Create several feedforwad layers with skip connections.
    for layer_idx in range(num_mlp_blocks):
        features = create_mlp(
            hidden_units=feedforward_units,
            dropout_rate=dropout_rate,
            activation=keras.activations.gelu,
            normalization_layer=layers.LayerNormalization(epsilon=1e-6),
            name=f"feedforward_{layer_idx}",
        )(features)

    # Compute MLP hidden_units.
    mlp_hidden_units = [
        factor * features.shape[-1] for factor in mlp_hidden_units_factors
    ]
    # Create final MLP.
    features = create_mlp(
        hidden_units=mlp_hidden_units,
        dropout_rate=dropout_rate,
        activation=keras.activations.selu,
        normalization_layer=layers.BatchNormalization(),
        name="MLP",
    )(features)

    # Add a sigmoid as a binary classifer.
    outputs = layers.Dense(units=1, activation="sigmoid", name="sigmoid")(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


baseline_model = create_baseline_model(
    embedding_dims=EMBEDDING_DIMS,
    num_mlp_blocks=NUM_MLP_BLOCKS,
    mlp_hidden_units_factors=MLP_HIDDEN_UNITS_FACTORS,
    dropout_rate=DROPOUT_RATE,
)

print("Total model weights:", baseline_model.count_params())
keras.utils.plot_model(baseline_model, show_shapes=True, rankdir="LR")

Total model weights: 9753
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


c:\jhs\miniconda\envs\jhs\Lib\site-packages\numpy\core\numeric.py:2463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [57]:
history = run_experiment(
    model=baseline_model,
    train_data_file=train_data_file,
    test_data_file=test_data_file,
    num_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    batch_size=BATCH_SIZE,
)

Start training the model...
Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'IteratorGetNext' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "c:\jhs\miniconda\envs\jhs\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\jhs\miniconda\envs\jhs\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "c:\jhs\miniconda\envs\jhs\Lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "c:\jhs\miniconda\envs\jhs\Lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\jhs\miniconda\envs\jhs\Lib\asyncio\base_events.py", line 607, in run_forever
      self._run_once()
    File "c:\jhs\miniconda\envs\jhs\Lib\asyncio\base_events.py", line 1922, in _run_once
      handle._run()
    File "c:\jhs\min